# Electric Arc Furnace

In [1]:
# Dependencies.
from IPython.display import HTML, Markdown

import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

from posted import TEDF


# Set variable of TEDF.
var = "Tech|Electric Arc Furnace"

# Loading the TEDF.
tedf = TEDF.load(var)

# Define units to use for energy carriers.
units={
    "Electricity": "MWh",
    "Heat": "MWh",
    "Natural Gas": "MWh_NG_LHV",
    "Coal": "MWh_coal_LHV",
}

## Fields

The techno-economic data is distinguished across the following additional fields.

### Operation Mode (`mode`)

In [2]:
Markdown(
    "\n".join(f"* **{code}**: {desc}" for code, desc in tedf.fields["mode"].codes.items())
)

* **Primary**: Operate with direct reduced iron (primary route).
* **Secondary**: Operate with scrap (secondary route).

### Reheating (`reheating`)

In [3]:
Markdown(
    "\n".join(f"* **{code}**: {desc}" for code, desc in tedf.fields["reheating"].codes.items())
)

* **w/ reheating**: Working with cold direct reduced iron that needs reheating.
* **w/o reheating**: Working with hot direct reduced iron that needs no reheating.

## Aggregated parameters

All data added to the POSTED database is aggregated automatically using the POSTED framework. The result yields the following parameters:

In [4]:
aggregated = tedf.aggregate(units=units, append_references=True)

display(
    aggregated
    .pivot(
        index=aggregated.columns[:-3],
        columns=["variable", "unit"],
        values="value",
    )
    .map(lambda x: float(f"{x:.3g}") if not pd.isnull(x) else x)
    .fillna("")
)

/home/runner/work/posted/posted/posted/noslag/mapping.py:50: POSTEDWarning:

No entry for `OCF` found when mapping `OPEX Fixed Specific` to `OPEX Fixed`. Assuming OCF of 100%.
        source       mode      reheating component             variable  \
99   Vogl-2018    Primary   w/ reheating    Labour  OPEX Fixed Specific   
106  Vogl-2018    Primary  w/o reheating    Labour  OPEX Fixed Specific   
113  Vogl-2018  Secondary   w/ reheating    Labour  OPEX Fixed Specific   
120  Vogl-2018  Secondary  w/o reheating    Labour  OPEX Fixed Specific   

      reference_variable      value  
99   Output|Steel Liquid  75.452709  
106  Output|Steel Liquid  75.452709  
113  Output|Steel Liquid  75.452709  
120  Output|Steel Liquid  75.452709  



Loading ITables v2.6.2 from the internet... (need help?)


## Energy demand

The figure below gives an overview of the energy demand reported by different sources across different energy carriers.

In [5]:
data_all_sources = tedf.aggregate(units=units, reheating="w/o reheating", agg="component")
data_agg_posted = tedf.aggregate(units=units, reheating="w/o reheating")

df_plot = (
    pd.concat([
        data_all_sources,
        data_agg_posted.assign(source="POSTED-default"),
    ])
    .assign(variable=lambda df: df["variable"].str.extract(r"^Input\|(.*)", expand=False))
    .query(f"variable.isin({list(units)})")
)

display(
    df_plot
    .plot.bar(
        x="source",
        y="value",
        color="variable",
        facet_col="mode",
    )
    .update_xaxes(
        title=None,
    )
    .for_each_annotation(
        lambda a: a.update(text=": ".join(a.text.split("=")))
    )
    .update_layout(
        legend_title=None,
        xaxis_title=None,
        yaxis_title="Energy demand per {reference_variable}  [ MWh / {reference_unit} ]".format(**df_plot.iloc[0]),
    )
)

/home/runner/work/posted/posted/posted/noslag/mapping.py:50: POSTEDWarning:

No entry for `OCF` found when mapping `OPEX Fixed Specific` to `OPEX Fixed`. Assuming OCF of 100%.
       source       mode component             variable   reference_variable  \
52  Vogl-2018    Primary    Labour  OPEX Fixed Specific  Output|Steel Liquid   
59  Vogl-2018  Secondary    Labour  OPEX Fixed Specific  Output|Steel Liquid   

        value  
52  75.452709  
59  75.452709  



/home/runner/work/posted/posted/posted/noslag/mapping.py:50: POSTEDWarning:

No entry for `OCF` found when mapping `OPEX Fixed Specific` to `OPEX Fixed`. Assuming OCF of 100%.
       source       mode component             variable   reference_variable  \
52  Vogl-2018    Primary    Labour  OPEX Fixed Specific  Output|Steel Liquid   
59  Vogl-2018  Secondary    Labour  OPEX Fixed Specific  Output|Steel Liquid   

        value  
52  75.452709  
59  75.452709  



## Raw data

In [6]:
Markdown(f"""
The table below contains the raw data contained in the public POSTED database. This data has not be automatically normalised or harmonised in any way. You can also find this data in the GitHub repo in this file:
{link_public_github(var)}
""")


The table below contains the raw data contained in the public POSTED database. This data has not be automatically normalised or harmonised in any way. You can also find this data in the GitHub repo in this file:
<a href="https://github.com/PhilippVerpoort/posted/blob/main/posted/database/tedfs/Tech/Electric Arc Furnace.csv">posted/database/tedfs/Tech/Electric Arc Furnace.csv</a>


In [7]:
edit(tedf)

Loading ITables v2.6.2 from the internet... (need help?)
